In [1]:
using Revise

In [2]:
using Flux
using ProgressMeter
using MLDataPattern
using JLD2
using Plots; gr()

Plots.GRBackend()

In [3]:
@load "../2018-05-08-box-atlas-longer-miqp/boxatlas.jld2" net

1-element Array{Symbol,1}:
 :net

In [4]:
net

Chain(Dense(22, 128, NNlib.elu), Dense(128, 128, NNlib.elu), Dense(128, 1))

In [8]:
c = net.layers[1]

Dense(22, 128, NNlib.elu)

In [11]:
params(net.layers[1])

2-element Array{Any,1}:
 param([1.22087 -2.53835 … -0.182239 0.896345; -3.26301 4.10768 … 0.423817 0.509198; … ; -2.26832 -3.15768 … 0.111558 -0.0575526; -7.51567 0.215953 … 0.203079 -0.494408])                                        
 param([-3.37196, -3.76363, -1.86752, -1.00524, -2.15658, -0.646821, 0.951493, -4.02201, 0.224544, -3.1805  …  -1.42739, -1.43818, 5.33455, -2.42154, -2.16359, -1.12987, -1.11951, -2.10481, 0.169921, -0.99396])

In [12]:
params(net.layers[2])

2-element Array{Any,1}:
 param([2.15374 1.62833 … -0.575029 0.351076; 1.39631 -2.76972 … 1.64741 0.428588; … ; 0.863245 -2.84132 … -3.09645 -3.86383; 1.39174 -0.518569 … -0.662874 0.5662])                                                   
 param([3.98693, -0.928702, -1.62388, -0.734205, -0.734243, 1.43787, -0.999678, -1.64648, -1.71407, -1.32597  …  -0.953853, -1.7636, -1.86719, -1.70832, -1.34158, -0.589168, -1.73559, -0.879125, -1.72201, -1.64609])

In [13]:
params(net.layers[3])

2-element Array{Any,1}:
 param([0.467793 2.03164 … 2.55718 -5.66191])
 param([3.9591])                             

In [14]:
params(sin)

0-element Array{Any,1}

In [ ]:
c.

In [42]:
function untrack(f::Function)
    @assert isempty(params(f))
    f
end

function untrack(d::Dense)
    W = Flux.Tracker.data(d.W)
    b = Flux.Tracker.data(d.b)
    σ = d.σ
    function f(x)
        y = W * x + b
        ReverseDiff.@forward(σ).(y)
    end
end

function untrack(chain::Chain)
    foldl(∘, reverse(untrack.(chain.layers)))
end

untrack (generic function with 3 methods)

In [43]:
n2 = untrack(net)

(::#55) (generic function with 1 method)

In [62]:
scalarize(f) = x -> f(x)[1]

scalarize (generic function with 1 method)

In [44]:
x = rand(22)
n2(x)

1-element Array{Float64,1}:
 902.638

In [45]:
net(x)

Tracked 1-element Array{Float64,1}:
 902.638

In [46]:
using ReverseDiff

In [49]:
n2(x)[]

902.6376139140335

In [66]:
@time ReverseDiff.gradient(scalarize(n2), x)

  0.000366 seconds (92 allocations: 328.734 KiB)


22-element Array{Float64,1}:
  206.847  
   49.6413 
 2316.79   
  120.139  
  -29.5312 
 -412.175  
  401.629  
  -23.3403 
   -3.99077
   51.9859 
  -41.9921 
 -194.188  
   36.7822 
  183.252  
   -4.10361
   11.7041 
  -29.1055 
    3.47161
   -1.76894
  -47.2386 
  -14.3045 
  -23.7481 

In [71]:
tape = ReverseDiff.HessianTape(scalarize(n2), x)
compiled = ReverseDiff.compile(tape)

ReverseDiff.CompiledTape(#24)

In [72]:
out = zeros(22, 22)
@time ReverseDiff.hessian!(out, compiled, x)

  0.248609 seconds (4 allocations: 160 bytes)


22×22 Array{Float64,2}:
  55.8038     -70.7569    134.248    …  -21.3573    -24.5013    -11.0298 
 -70.7569     -47.3807    174.44         27.5421    -29.2959    -12.2987 
 134.248      174.44     -509.34        -30.0504     64.3747     39.1034 
 -82.4694    -128.964     257.815        14.0759     -8.86561    -5.6386 
  16.3895     -31.8498     73.3176       -1.39876    -7.38633    -0.49231
  -0.297829    20.7295    -44.064    …    9.29376    -1.87298     4.46543
 -23.0033     -47.9946    166.57         11.8059    -15.4254     -6.24294
  49.3122      16.9145    -53.7816       -2.31199    -2.39223     7.03422
   8.28068      3.06046   -13.6633       -6.98614     1.98829    -1.49632
  13.2115       1.89937   -30.8324        0.833824    5.35581     7.14559
 -14.9793     -11.5063     52.4468   …    6.76313   -13.4739     -5.86812
 -41.8388     -36.921     133.082         4.43358   -17.2245    -15.8846 
 -14.3772      18.5794    -21.5629       10.1174      1.96074     2.89931
 -55.2706     

In [73]:
using ForwardDiff

In [77]:
f = scalarize(n2)

(::#24) (generic function with 1 method)

In [85]:
using BenchmarkTools

INFO: Precompiling module BenchmarkTools.


In [88]:
out = zeros(22, 22)
A = rand(22, 22)
b = rand()
@btime $out .= $A .* $b

  171.232 ns (0 allocations: 0 bytes)


22×22 Array{Float64,2}:
 0.0925297  0.0712245  0.506714    …  0.105502    0.0451499  0.146562  
 0.161347   0.356692   0.539082       0.571787    0.303389   0.090163  
 0.442811   0.562144   0.50791        0.640467    0.345442   0.222742  
 0.0445345  0.209675   0.185431       0.51085     0.637026   0.317615  
 0.308315   0.344108   0.55604        0.530137    0.182197   0.0452428 
 0.369948   0.0620466  0.535653    …  0.0369485   0.286301   0.262048  
 0.0125953  0.0757633  0.257421       0.0335656   0.0987677  0.602888  
 0.417654   0.183318   0.00210737     0.0387144   0.0691739  0.135203  
 0.49123    0.306442   0.200245       0.409599    0.489434   0.00407723
 0.267733   0.17738    0.0557016      0.285967    0.266881   0.635272  
 0.568614   0.516724   0.447967    …  0.00436033  0.372324   0.0337528 
 0.345052   0.612714   0.348469       0.284681    0.59234    0.582042  
 0.648334   0.603048   0.418363       0.210825    0.225154   0.373626  
 0.633754   0.589234   0.247976       0.

In [87]:
@btime $(rand(22, 22)) * $(randn())

  477.174 ns (1 allocation: 3.94 KiB)


22×22 Array{Float64,2}:
 -0.145755   -0.684377  -0.445198   …  -0.137079   -0.101266    -0.114717 
 -0.496101   -0.469564  -0.588811      -0.412051   -0.250028    -0.135832 
 -0.616456   -0.214906  -0.672953      -0.811843   -0.0116028   -0.386744 
 -0.18769    -0.203996  -0.34569       -0.818978   -0.43519     -0.323192 
 -0.472893   -0.550577  -0.457573      -0.563912   -0.538244    -0.1876   
 -0.37347    -0.146596  -0.0942543  …  -0.751463   -0.0940785   -0.2415   
 -0.649897   -0.302357  -0.563572      -0.208079   -0.888795    -0.0452466
 -0.500663   -0.610137  -0.44453       -0.0100581  -0.282452    -0.186063 
 -0.918381   -0.668095  -0.670962      -0.527256   -0.131696    -0.110296 
 -0.427512   -0.287439  -0.108917      -0.44958    -0.180058    -0.201135 
 -0.780857   -0.499923  -0.721426   …  -0.051894   -0.30944     -0.786901 
 -0.0623661  -0.102748  -0.364184      -0.77077    -0.791102    -0.124904 
 -0.67136    -0.310799  -0.791655      -0.45308    -0.164049    -0.317112 
 

In [83]:
cfg = ForwardDiff.HessianConfig(f, x);
result = zeros(22, 22)
@time ForwardDiff.hessian!(result, f, x, cfg)

  0.036424 seconds (153 allocations: 4.306 MiB)


22×22 Array{Float64,2}:
  55.8038     -70.7569    134.248    …  -21.3573    -24.5013    -11.0298 
 -70.7569     -47.3807    174.44         27.5421    -29.2959    -12.2987 
 134.248      174.44     -509.34        -30.0504     64.3747     39.1034 
 -82.4694    -128.964     257.815        14.0759     -8.86561    -5.6386 
  16.3895     -31.8498     73.3176       -1.39876    -7.38633    -0.49231
  -0.297829    20.7295    -44.064    …    9.29376    -1.87298     4.46543
 -23.0033     -47.9946    166.57         11.8059    -15.4254     -6.24294
  49.3122      16.9145    -53.7816       -2.31199    -2.39223     7.03422
   8.28068      3.06046   -13.6633       -6.98614     1.98829    -1.49632
  13.2115       1.89937   -30.8324        0.833824    5.35581     7.14559
 -14.9793     -11.5063     52.4468   …    6.76313   -13.4739     -5.86812
 -41.8388     -36.921     133.082         4.43358   -17.2245    -15.8846 
 -14.3772      18.5794    -21.5629       10.1174      1.96074     2.89931
 -55.2706     

In [76]:
@time ForwardDiff.hessian(scalarize(n2), x)

  0.026935 seconds (161 allocations: 4.330 MiB)


22×22 Array{Float64,2}:
  55.8038     -70.7569    134.248    …  -21.3573    -24.5013    -11.0298 
 -70.7569     -47.3807    174.44         27.5421    -29.2959    -12.2987 
 134.248      174.44     -509.34        -30.0504     64.3747     39.1034 
 -82.4694    -128.964     257.815        14.0759     -8.86561    -5.6386 
  16.3895     -31.8498     73.3176       -1.39876    -7.38633    -0.49231
  -0.297829    20.7295    -44.064    …    9.29376    -1.87298     4.46543
 -23.0033     -47.9946    166.57         11.8059    -15.4254     -6.24294
  49.3122      16.9145    -53.7816       -2.31199    -2.39223     7.03422
   8.28068      3.06046   -13.6633       -6.98614     1.98829    -1.49632
  13.2115       1.89937   -30.8324        0.833824    5.35581     7.14559
 -14.9793     -11.5063     52.4468   …    6.76313   -13.4739     -5.86812
 -41.8388     -36.921     133.082         4.43358   -17.2245    -15.8846 
 -14.3772      18.5794    -21.5629       10.1174      1.96074     2.89931
 -55.2706     